In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')

    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews

    !pip install joypy

    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

# AE Model

In [3]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"

model_name = f"{root}/Zamuro/temporal_zamuro/models/model_{model_type}_{identifier}_{date_format}_final.pth"
config = torch.load(f'{root}/Zamuro/temporal_zamuro/configs/config_{model_type}_{identifier}_{date_format}.pth', map_location=torch.device('cpu'))
model = AE(num_hiddens=config["num_hiddens"]).to(device)
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

# y = torch.load(f"temporal_zamuro/Features/{folder}/AE_labels_{date_format}_No_rain_Audios_Jaguas.pth",  map_location=torch.device('cpu'))
X = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))
path_flat = [item for sublist in y_path for item in sublist]
path_flat = np.asarray(path_flat)
print(X.shape)

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
Normalizer_ = Normalizer().fit(X)
X_norm = Normalizer_.transform(X)
PCA_ = PCA(n_components=60).fit(X_norm)
X_PCA = PCA_.transform(X_norm)
# X_TSNE = TSNE(n_components=60, learning_rate="auto", init='random', random_state=0).fit_transform(X_PCA)
reducer = umap.UMAP(min_dist=0.9, n_components=60)
X_UMAP = reducer.fit_transform(X_norm)
X_batch = np.reshape(X_UMAP, (X_UMAP.shape[0]//5,5,X_UMAP.shape[1]))
# X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

torch.Size([325320, 5184])


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/Audios_Zamuro/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"
root_ai_Zamuro = f"{root}/Zamuro/AI_Caract/AI_Zamuro.csv"
root_vggish_Zamuro = f"{root}/Zamuro/vggish/Features_vggish"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)
ai = pd.read_csv(root_ai_Zamuro, index_col=0)
ai.dropna(inplace=True)
ai.set_index("file",inplace=True, drop=True)
ai.drop(columns=["Date"], inplace=True)
def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)

### VGGish data

In [6]:
X_vgg = torch.load(f"{root_vggish_Zamuro}/vggish_features.pth")
y_vgg = torch.load(f"{root_vggish_Zamuro}/vggish_filenames.pth")
y_vgg2 = [i.split('/')[6:8] for i in y_vgg]

for i in range(len(y_vgg2)):
    y_vgg2[i] = f"{y_vgg2[i][0]}_{y_vgg2[i][1]}"
X_vgg2 = X_vgg
X_vgg3 = []
remove = []
remove_idx = []
keep_idx = []

for i in range(len(y_vgg2)):
    #a[i] = X_vgg2[i]
    if y_vgg2[i] not in y_3:
        remove.append(y_vgg2[i])
        remove_idx.append(X_vgg2[i])
    else:
        keep_idx.append(i)
        
for i in range(len(remove)): 
    y_vgg2.remove(remove[i])
    
for i in keep_idx:
    X_vgg3.append(X_vgg2[i])
    
X_vgg4 = np.zeros([64543,62,128])
for i in range (len(X_vgg3)):
    X_vgg4[i] = X_vgg3[i].numpy()
X_vgg4 = X_vgg4.reshape(X_vgg4.shape[0], X_vgg4.shape[1]*X_vgg4.shape[2])
labels = []
for i in range(len(y_vgg2)):
    labels.append(audios.loc[y_vgg2[i], "cover"])
    
from sklearn.model_selection import train_test_split
X_train, X_test, labels_train, labels_test = train_test_split(X_vgg4, labels, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=16, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

Accuracy: 0.685025950886978
f1: 0.5364640981268275
recall 0.519688783160868


### Acoustic Indices Data

In [5]:
y_ = np.asarray(y_path)
y_2 = y_[:,0]
for i in range(len(y_2)):
    y_2[i] = y_2[i][0:-2] 
y_3 = list(y_2)

X_ai = []
remove = []
s=0
for i in range(len(y_3)):
    
    try:
        X_ai.append(ai.loc[y_3[i]])
    except:
        s+=1
        remove.append(y_3[i])
for i in range(len(remove)):
    y_3.remove(remove[i])
X_ai = np.asarray(X_ai)

labels_ai = []
# audios.set_index("Filename", inplace=True)
for i in range(len(y_3)):
    labels_ai.append(audios.loc[y_3[i], "cover"])
    
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ai, labels_ai, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf_rf = RandomForestClassifier(max_depth=16, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

Accuracy: 0.7948717948717948
f1: 0.7283220250028491
recall 0.6986985474605927


### Autoencoders Features and Labels using independent segments

In [7]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# audios.set_index("Filename", inplace=True)
labels = []
for i in range(len(path_flat)):
    labels.append(audios.loc[path_flat[i][0:-2], "cover"])
    
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=16, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

Accuracy: 0.8179638509774991
f1: 0.7591613503594904
recall 0.7300207327387304


### Autoencoders Features and Labels using 5 segments of the audio

In [ ]:
X_c = X_UMAP
y_c = list(path_flat)

In [ ]:
guard = X_c


In [8]:
X_c = X_UMAP
y_c = list(path_flat)
print(len(y_c))

# for i in range(len(y_c)):
#     split = y_c[i].split("_")[0:3]
#     y_c[i] = f"{split[0]}_{split[1]}_{split[2]}"

remove_2 = []
counter = 0
for i in range(len(remove)):
    for j in range(1,6):
        counter+=1
        a = np.where(y_c == f"{remove[i]}_{j}")
        remove_2.append(a[0][0])
#         y_c.remove(f"{remove[i]}_{j}")
#         X_c = np.delete(X_c, i, axis=0)
            
y_c = np.delete(y_c, remove_2, 0)
X_c = np.delete(X_c, remove_2,0)

X_batch = np.reshape(X_c, (X_c.shape[0]//5,5,X_c.shape[1]))
y_path = np.reshape(y_c, (y_c.shape[0]//5, 5))

325320


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_batch, y_path, test_size=0.2,random_state=0)

X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

labels_train = []
for i in range(len(y_train)):
    labels_train.append(audios.loc[y_train[i][0:-2], "Habitat"])
labels_test = []
for i in range(len(y_test)):
    labels_test.append(audios.loc[y_test[i][0:-2], "Habitat"])